# Basic Recommendation on Movielens Dataset

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from enum import auto, Enum
from datetime import datetime
from collections import defaultdict
from timeit import default_timer
from collections import defaultdict
import random
%matplotlib inline
sns.set_style('white')

## Load Dataset

In [2]:
class PredictionTimeConstraint(Enum):
    AT = auto()
    IN = auto()
    NO = auto()
    
class RecSys:
    def __init__(self, movie_ratings_data, min_common_elements):
        self.movie_ratings = movie_ratings_data
        self.min_common_elements = min_common_elements
        # User Movie Matrix, Rows-> Movie_Titles :: Columns -> User_IDs :: Values -> User Ratings
        self.user_movie_matrix = self.movie_ratings.pivot_table(index='title', columns='user_id', values='rating')
        self.user_corr_matrix = self.user_movie_matrix.corr(method='pearson', min_periods=min_common_elements)
        
        # Temporal User Correlations Matrix CACHE details
        # Since analysis requires us do more than one predictions with same dates save the user_corr_matrix for performance
        self.temporal_user_corr_matrix = None
        self.temporal_time_constraint = None
        self.temporal_dt = None
        self.temporal_start_dt = None
        self.temporal_end_dt = None
    
    def get_k_neighbours(self, user_id, k = 20, 
                         time_constraint=None,
                         start_dt=None,
                         end_dt=None,
                         dt=None):
        
        # Apply Time Constraint and get user_correlations to each other as matrix
        if time_constraint is None:
            user_corr_matrix = self.user_corr_matrix
        else:
            user_corr_matrix = self.create_temporal_corr_matrix(time_constraint=time_constraint,
                                                           dt=dt, start_dt=start_dt, end_dt=end_dt)
        
        # Exit if matrix is None
        if user_corr_matrix is None:
            return None
        
        # Get the 'user_id's sorrelations
        user_corrs = user_corr_matrix.get(user_id)
        if user_corrs is None:
            return None

        # Drop any null, if found
        user_corrs.dropna(inplace=True)

        # Create A dataframe from not-null correlations of the 'user_id'
        users_alike = pd.DataFrame(user_corrs)

        # Rename the only column to 'correlation'
        users_alike.columns= ['correlation']

        # Sort the user correlations in descending order 
        #     so that first one is the most similar, last one least similar
        users_alike.sort_values(by='correlation', ascending=False, inplace=True)

        # Eliminate Correlation to itself by deleting first row, 
        #     since biggest corr is with itself it is in first row
        return users_alike.iloc[1:k]
    
    def predict_movie(self, user_id, movie_id, k = 10, 
                      time_constraint=None,
                      start_dt=None,
                      end_dt=None,
                      dt=None):
        
        # If a movie with movie_id not exists, predict 0
        if self.movie_ratings.loc[self.movie_ratings['item_id'] == movie_id] is None:
            print("Movie Not Found!!")
            return 0

        # Get Nearest Neighbours of the 'user_id'
        k_nearest_neighbours = self.get_k_neighbours(user_id, k=k, 
                                                     time_constraint=time_constraint,
                                                     start_dt=start_dt, end_dt=end_dt,
                                                     dt=dt)

        # Calculate Mean Centered Prediciton
        user_avg_rating = self.get_user_avg_rating(user_id)
        
        #########################################################################
        ##### IN CASE WE COULDNT FIND ENOUGH NEIGHBOURS, PREDICT AS HIS AVERAGE
        if k_nearest_neighbours is None or k_nearest_neighbours.empty:
            return 0 #user_avg_rating
        #########################################################################
        weighted_sum = 0.0
        sum_of_weights = 0.0
        for neighbour_id, data in k_nearest_neighbours.iterrows():
            # Get each neighbour's correlation 'user_id' and her rating to 'movie_id' 
            neighbour_corr = data['correlation']
            neighbour_data = self.movie_ratings.loc[(self.movie_ratings['user_id'] == neighbour_id) & (self.movie_ratings['item_id'] == movie_id)]
            # If the neighbour doesnt give rating to the movie_id, pass this around of the loop
            if neighbour_data.empty:
                continue
            neighbour_avg_rating = self.get_user_avg_rating(neighbour_id)
            neighbour_rating = float(neighbour_data.rating)
            neighbour_mean_centered_rating =  neighbour_rating - neighbour_avg_rating
            # Calculate Weighted sum and sum of weights
            weighted_sum += neighbour_mean_centered_rating * neighbour_corr
            sum_of_weights += neighbour_corr

        # Predict
        if sum_of_weights != 0:
            prediction_rating = user_avg_rating + (weighted_sum / sum_of_weights)
        else:
            #########################################################################
            ##### IN CASE WE COULDNT FIND ENOUGH NEIGHBOURS THAT GIVEN RATING TO THE movie_id, PREDICT AS HIS AVERAGE
            prediction_rating = 0 #user_avg_rating
            #########################################################################
            
        return prediction_rating
    
    def is_temporal_cache_valid(self,
                                time_constraint=None,
                                start_dt=None,
                                end_dt=None,
                                dt=None):
        
        if self.temporal_time_constraint != time_constraint or time_constraint is None:
            return False
        
        # If 'AT' constraint found and same as last calculated matrix constraints then cache valid
        if time_constraint == PredictionTimeConstraint.AT and self.temporal_dt == dt:
            return True
        
        # If 'IN' constraint found and same as last calculated matrix constraints then cache valid
        if time_constraint == PredictionTimeConstraint.IN and self.temporal_start_dt == start_dt and self.temporal_end_dt == end_dt:
            return True

        # Otherwise cache invalid
        return False
    
    def create_temporal_corr_matrix(self, time_constraint=PredictionTimeConstraint.NO,
                                          start_dt=None,
                                          end_dt=None,
                                          dt=None):
        
        # Check cache, if valid return the cache
        temporal_cache_status = self.is_temporal_cache_valid(time_constraint=time_constraint,start_dt=start_dt,end_dt=end_dt,dt=dt)
        if temporal_cache_status == True:
            return self.temporal_user_corr_matrix
        
        # No valid cache, recreate the correlations matrix 
        if time_constraint == PredictionTimeConstraint.AT and dt != None:
            user_movie_matrix = self.movie_ratings[(self.movie_ratings.timestamp < dt)].pivot_table(index='title', columns='user_id', values='rating') 
        elif time_constraint == PredictionTimeConstraint.IN and start_dt != None and end_dt != None:
            user_movie_matrix =  self.movie_ratings[(self.movie_ratings.timestamp >= start_dt) & (self.movie_ratings.timestamp < end_dt)].pivot_table(index='title', columns='user_id', values='rating') 
        else:
            return None
        
        # Create CACHE
        self.temporal_time_constraint = time_constraint
        self.temporal_start_dt = start_dt
        self.temporal_end_dt = end_dt
        self.temporal_dt = dt
        self.temporal_user_corr_matrix = user_movie_matrix.corr(method='pearson', min_periods=self.min_common_elements) 
        
        return self.temporal_user_corr_matrix
        
    def get_user_rating(self, user_id):
        return self.movie_ratings.loc[self.movie_ratings['user_id'] == user_id]

    def get_user_rating_for_movie(self, user_id, movie_id):
        try:
            rating = self.movie_ratings.loc[ (self.movie_ratings['user_id'] == user_id) & (self.movie_ratings['item_id'] == movie_id) ].values[0,2]
        except IndexError:
            rating = 0
        return rating 

    def get_rating_timestamp(self, user_id, movie_id):
        return self.movie_ratings.loc[ (self.movie_ratings['user_id'] == user_id) & (self.movie_ratings['item_id'] == movie_id) ].values[0,3]

    def get_movies_watched(self, user_id, year = 2015, month = 7, day = 15):
        return self.movie_ratings.loc[((self.movie_ratings['user_id'] == user_id)) & (self.movie_ratings.timestamp < datetime(year, month, day))][['item_id','title','rating']].set_index('item_id')

    def get_user_avg_rating(self, user_id):
        return self.movie_ratings.loc[((self.movie_ratings['user_id'] == user_id))].rating.mean()

    def get_user_avg_rating_at(self, user_id, year = 2015, month = 7, day = 15):
        return self.movie_ratings.loc[((self.movie_ratings['user_id'] == user_id)) & (self.movie_ratings.timestamp < datetime(year, month, day))].rating.mean()
    
    
    def get_movies_watched(self, user_id,
                           time_constraint=None, 
                           dt=None, start_dt=None, end_dt=None):
        if time_constraint == None or time_constraint == PredictionTimeConstraint.NO:
            return self.movie_ratings.loc[(self.movie_ratings['user_id'] == user_id)][['item_id','rating']]
        elif time_constraint == PredictionTimeConstraint.AT and dt != None:
            return self.movie_ratings.loc[(self.movie_ratings['user_id'] == user_id) & (self.movie_ratings.timestamp < dt)][['item_id','rating']]
        elif time_constraint == PredictionTimeConstraint.IN and start_dt != None and end_dt != None:
            return self.movie_ratings.loc[(self.movie_ratings['user_id'] == user_id) & (self.movie_ratings.timestamp >= start_dt) & (self.movie_ratings.timestamp < end_dt)][['item_id','rating']]
        else:
            return None                                      # Unknown time constraint, exit the function
    
    def get_random_movie_watched(self, user_id):
        return self.movie_ratings.loc[(self.movie_ratings['user_id'] == user_id)][['item_id']].iloc[0][0]
    
    def predict_movies_watched(self, user_id,
                               n = 10,
                               k = 10,
                               time_constraint=None,
                               dt = None,
                               start_dt = None,
                               end_dt = None):
        
        # Get all movies watched by a user
        movies_watched = self.get_movies_watched(user_id)

        if movies_watched is None or movies_watched.empty:
            return None

        predictions = list()
        
        number_of_predictions = 0
        for _,row in movies_watched.iterrows():
            prediction = self.predict_movie(user_id=user_id, movie_id=row['item_id'], 
                                              time_constraint=time_constraint, k=k,
                                              start_dt=start_dt, end_dt=end_dt,dt=dt)
            if number_of_predictions == n:
                break
            predictions.append([prediction,row['rating'],row['item_id']])
            number_of_predictions += 1

        predictions_df = pd.DataFrame(predictions, columns = ['prediction','rating', 'movie_id'])
        predictions_df.movie_id = predictions_df.movie_id.astype(int)
        return predictions_df.set_index('movie_id')
    
    ## Load Datasets
    @staticmethod
    def load_movielens_small():
        col_names = ['user_id', 'item_id', 'rating', 'timestamp']
        ratings = pd.read_csv(r'C:\Users\Yukawa\datasets\ml-latest-small\ratings.csv', sep=',', header=1, names=col_names)

        col_names = ['item_id', 'title', 'genres']
        movies = pd.read_csv(r'C:\Users\Yukawa\datasets\ml-latest-small\movies.csv', sep=',', header=0, names=col_names)
        
        movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
        movies.year = pd.to_datetime(movies.year, format='%Y')
        movies.year = movies.year.dt.year # As there are some NaN years, resulting type will be float (decimals)
        movies.title = movies.title.str[:-7]

        ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s', origin='unix')

        movie_ratings = pd.merge(ratings, movies, on='item_id')

        return movie_ratings
    
    ## Accuracy Metrics
    @staticmethod
    def rmse(predictions_df):
        number_of_predictions = predictions_df.count()
        sum_of_square_differences = 0.0
        for _,row in predictions_df.iterrows():
            prediction_rating = row['prediction'] 
            if prediction_rating != 0:
                sum_of_square_differences += (row['rating'] - prediction_rating) ** 2
        result = sum_of_square_differences/number_of_predictions
        return result[0]

In [3]:
movie_ratings = RecSys.load_movielens_small()

In [4]:
movie_ratings.head()

,user_id,item_id,rating,timestamp,title,genres,year
0,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men,Comedy|Romance,1995.0
1,6,3,5.0,1996-10-17 12:11:36,Grumpier Old Men,Comedy|Romance,1995.0
2,19,3,3.0,2000-08-08 04:07:16,Grumpier Old Men,Comedy|Romance,1995.0
3,32,3,3.0,1997-02-23 22:16:12,Grumpier Old Men,Comedy|Romance,1995.0
4,42,3,4.0,2001-07-27 08:04:05,Grumpier Old Men,Comedy|Romance,1995.0


## Most Active Users

In [5]:
active_user_ratings = pd.DataFrame(movie_ratings.groupby('user_id')['rating'].mean())
active_user_ratings['No_of_ratings'] = pd.DataFrame(movie_ratings.groupby('user_id')['rating'].count())
active_user_ratings.sort_values(by=['No_of_ratings'], ascending=False).head(10)

,rating,No_of_ratings
user_id,,
414,3.391957,2698
599,2.642050,2478
474,3.398956,2108
448,2.847371,1864
274,3.235884,1346
610,3.688556,1302
68,3.233730,1260
380,3.673235,1218
606,3.657399,1115


In [6]:
%reset_selective -f (^active_user_ratings$)

## Evaluator

In [7]:
evaluator = RecSys(movie_ratings_data=movie_ratings, min_common_elements=5)

In [8]:
evaluator.predict_movies_watched(user_id=414, k=10)

,prediction,rating
movie_id,,
3,0.000000,4.0
6,0.000000,3.0
47,4.166467,4.0
50,4.206210,5.0
101,0.000000,4.0
110,3.687791,5.0
151,0.000000,5.0
157,0.000000,4.0
163,3.688742,4.0


In [9]:
RecSys.rmse(evaluator.predict_movies_watched(user_id=414, k=10))

0.2476589017082173

In [10]:
evaluator.predict_movie(449, 3, time_constraint=PredictionTimeConstraint.AT, dt=datetime(2015,7,15))

2.3876879699248117

In [11]:
evaluator.predict_movie(user_id=414, movie_id=3, k=20, time_constraint=PredictionTimeConstraint.IN, start_dt=datetime(2000,7,15), end_dt=datetime(2018,7,15))

3.0239916372236606

In [12]:
evaluator.predict_movie(user_id=414,movie_id=3, k=50,time_constraint=PredictionTimeConstraint.IN, start_dt=datetime(2000,7,15), end_dt=datetime(2018,7,15))

2.987779546119328

In [13]:
evaluator.predict_movies_watched(user_id=182, k=50, 
                       time_constraint=PredictionTimeConstraint.IN,
                       start_dt=datetime(2000,5,12), end_dt=datetime(2005,1,15))

,prediction,rating
movie_id,,
6,3.850931,4.5
47,3.993664,4.0
50,4.430692,4.5
70,2.772934,4.5
110,4.196354,3.5
163,3.696871,3.5
223,3.836557,4.5
231,3.259759,0.5
235,4.599588,4.5


In [14]:
RecSys.rmse(evaluator.predict_movies_watched(user_id=182, k=50, 
                       time_constraint=PredictionTimeConstraint.IN,
                       start_dt=datetime(2000,5,12), end_dt=datetime(2005,1,15)))

1.3078596127156936

In [15]:
RecSys.rmse(evaluator.predict_movies_watched(user_id=182, k=100,                       
                       time_constraint=PredictionTimeConstraint.IN,
                       start_dt=datetime(2000,5,12), end_dt=datetime(2005,1,15)))

1.3157696554561307

In [16]:
RecSys.rmse(evaluator.predict_movies_watched(user_id=182, k=1000,                       
                       time_constraint=PredictionTimeConstraint.IN,
                       start_dt=datetime(2000,5,12), end_dt=datetime(2005,1,15)))

1.3545725301140958

## Some Helper Functions

In [17]:
def get_random_n_users(n):
    users = list()
    for i in range(n):
        users.append(random.randint(1,610))
    return users

In [18]:
get_random_n_users(10)

[559, 137, 138, 504, 133, 388, 370, 282, 295, 506]

In [19]:
def get_random_movie_per_user(user_list):
    user_movie_list = list()
    for user_id in user_list:
        movie_id = evaluator.get_random_movie_watched(user_id)
        user_movie_list.append((user_id,movie_id))
    return user_movie_list

In [20]:
get_random_movie_per_user(get_random_n_users(10))

[(44, 3),
 (571, 70),
 (44, 3),
 (359, 296),
 (201, 260),
 (478, 1073),
 (592, 6),
 (104, 47),
 (304, 110),
 (272, 1240)]

In [21]:
def RMSE(predictions):
    number_of_predictions = len(predictions)
    sum_of_square_differences = 0.0
    for prediction,actual in predictions:
        sum_of_square_differences += (actual - prediction) ** 2
    result = sum_of_square_differences/number_of_predictions
    return result

## Compare 'AT Constraint' and 'NO Constraint' on time interval

In [22]:
def find_best_year_constraint(n, rmse_diff_limit, max_year_limit = 2016):
    min_year = movie_ratings['timestamp'].min().year + 5 ## No need to scan first 5 years, so less ratings, no meaning of filter
    max_year = max_year_limit # datetime.now().year
    
    user_list = get_random_n_users(n)
    
    winner_year = min_year
    winner_vote_count = 0

    # For Each Year
    for year in range(min_year, max_year):
        vote_count = 0                          # Keep track of votes per year, the one who gets most, wins
        for user_id in user_list:
            rmse_no_constraint = RecSys.rmse(evaluator.predict_movies_watched(user_id=user_id, k=10))
            rmse_with_constraint = RecSys.rmse(evaluator.predict_movies_watched(user_id=user_id, k=10, dt=datetime(year,1,15),
                                                                   time_constraint=PredictionTimeConstraint.AT))
            if abs(rmse_no_constraint - rmse_with_constraint) <= rmse_diff_limit:
                vote_count += 1
        
        if vote_count > winner_vote_count:
            winner_year = year
            winner_vote_count = vote_count
            
    return winner_year

In [31]:
find_best_year_constraint(20,0.1)

2014

In [24]:
def compare_no_time_constraint_and_at_constraint(number_of_users, year_constraint=2006, k=10):
    """
    For Random 'number_of_users' Users
    Calculate rmse without any time constraint
    Calculate rmse with time constraint(assuming the system is at year_constraint)
    return as dataframe and also include runtime's of the calculations
    """
    results = list()    
    for i in range(number_of_users):
        user_id = random.randint(1,610)
        
        st = default_timer()
        rmse_no_constraint = RecSys.rmse(evaluator.predict_movies_watched(user_id=user_id, k=k))
        runtime_no_constraint = default_timer() - st
        
        st = default_timer()
        rmse_with_constraint = RecSys.rmse(evaluator.predict_movies_watched(user_id=user_id, k=k,
                                                                       time_constraint=PredictionTimeConstraint.AT,
                                                                       dt=datetime(year_constraint,7,15)))
        runtime_with_constraint = default_timer() - st
        
        results.append([user_id,rmse_no_constraint,runtime_no_constraint,rmse_with_constraint,runtime_with_constraint])
    results = pd.DataFrame(results)
    results.columns= ['user_id','rmse','runtime1','temporal_rmse','runtime2']
    results.set_index('user_id', inplace=True)
    return results

In [25]:
result = compare_no_time_constraint_and_at_constraint(50, year_constraint=2006, k = 20)
print(f"No Constraint->> mean_rmse = {result['rmse'].mean()} mean_runtime = {result['runtime1'].mean()}")
print(f"With Constraint->> mean_rmse = {result['temporal_rmse'].mean()} mean_runtime = {result['runtime2'].mean()}")

No Constraint->> mean_rmse = 0.37322648607740594 mean_runtime = 0.27267731400000117
With Constraint->> mean_rmse = 0.2119634432306624 mean_runtime = 0.17082012400000038


In [26]:
result2 = compare_no_time_constraint_and_at_constraint(50, year_constraint=2011, k = 20)
print(f"No Constraint->> mean_rmse = {result2['rmse'].mean()} mean_runtime = {result2['runtime1'].mean()}")
print(f"With Constraint->> mean_rmse = {result2['temporal_rmse'].mean()} mean_runtime = {result2['runtime2'].mean()}")

No Constraint->> mean_rmse = 0.4172938483153468 mean_runtime = 0.26994572200000166
With Constraint->> mean_rmse = 0.2687560722184044 mean_runtime = 0.1992527800000005


In [27]:
result3 = compare_no_time_constraint_and_at_constraint(50, year_constraint=2016, k = 20)
print(f"No Constraint->> mean_rmse = {result3['rmse'].mean()} mean_runtime = {result3['runtime1'].mean()}")
print(f"With Constraint->> mean_rmse = {result3['temporal_rmse'].mean()} mean_runtime = {result3['runtime2'].mean()}")

No Constraint->> mean_rmse = 0.34375923887986615 mean_runtime = 0.2655628620000002
With Constraint->> mean_rmse = 0.309947279911403 mean_runtime = 0.28614980000000145


## Compare Periods and Their Performance

In [28]:
def compare_decay_periods(n,k=10):
    min_year = movie_ratings['timestamp'].min().year
    max_year = datetime.now().year
    
    user_list = get_random_n_users(n)
    user_movie_pair_list = get_random_movie_per_user(user_list)
    
    # Calculate RMSE for each period in between [2,10)
    for period in range(2,10):
        curr_year = min_year
        predictions = list()
        start_dt = default_timer()
        while (curr_year+period) < max_year:
            for user_id, movie_id in user_movie_pair_list:
                prediction = evaluator.predict_movie(user_id=user_id, movie_id=movie_id, k=k,
                                                     start_dt=datetime(curr_year,1,15),
                                                     end_dt = datetime(curr_year+period,1,15),
                                                     time_constraint=PredictionTimeConstraint.IN)
                if prediction != 0:
                    rating = evaluator.get_user_rating_for_movie(user_id=user_id,movie_id=movie_id)
                    predictions.append((prediction, rating))
                    #print(f"Period={period} interval=({curr_year},{curr_year+period}) prediction={prediction} actual={rating} user_id={user_id} movie_id={movie_id}")
            curr_year += period
        runtime = default_timer() - start_dt
        period_rmse = RMSE(predictions)
        print(f"\nPeriod={period} RMSE = {period_rmse} RunTime(Seconds) = {runtime}\n")

In [29]:
compare_decay_periods(n=100, k=10)


Period=2 RMSE = 0.8986219103445714 RunTime(Seconds) = 3.1235650999999933


Period=3 RMSE = 0.7000263906642337 RunTime(Seconds) = 2.4153676000000246


Period=4 RMSE = 0.7189399140504598 RunTime(Seconds) = 2.1995794000000046


Period=5 RMSE = 0.8321561996599401 RunTime(Seconds) = 2.3268416999999886


Period=6 RMSE = 0.6600208080248002 RunTime(Seconds) = 1.863307600000013


Period=7 RMSE = 0.8477324474598655 RunTime(Seconds) = 2.380570399999982


Period=8 RMSE = 0.5894127316652268 RunTime(Seconds) = 1.8260721000000046


Period=9 RMSE = 0.5820787874849106 RunTime(Seconds) = 2.058698700000008



In [30]:
compare_decay_periods(n=600, k=10)


Period=2 RMSE = 1.7504898652236385 RunTime(Seconds) = 17.030944600000026


Period=3 RMSE = 0.8534362663896442 RunTime(Seconds) = 12.856807599999996


Period=4 RMSE = 0.9382868167726545 RunTime(Seconds) = 11.108828500000016


Period=5 RMSE = 1.5642924844864363 RunTime(Seconds) = 9.879565899999989


Period=6 RMSE = 0.9007209800788105 RunTime(Seconds) = 8.904547699999995


Period=7 RMSE = 0.6623852289158911 RunTime(Seconds) = 10.083040299999993


Period=8 RMSE = 1.9050051248162538 RunTime(Seconds) = 7.65882910000002


Period=9 RMSE = 1.718598854482408 RunTime(Seconds) = 8.018100600000025

